In [43]:
import pandas as pd
from scipy.stats import t,ttest_ind,ks_2samp,shapiro,anderson 
import scipy.stats as stats

In [3]:
df_raw = pd.read_excel('619702 - Innovation at Uber - The Launch of Express POOL, Spreadsheet Supplement.xlsx', sheet_name ='Switchbacks' )

In [4]:
df_raw.head()

,city_id,period_start,wait_time,treat,commute,trips_pool,trips_express_pool,rider_cancellations,total_driver_payout,total_matches,total_double_matches
0,Boston,2018-02-19 07:00:00,2 mins,False,True,1415,3245,256,34458.411634,3372,1476
1,Boston,2018-02-19 09:40:00,5 mins,True,False,1461,2363,203,29764.349821,2288,1275
2,Boston,2018-02-19 12:20:00,2 mins,False,False,1362,2184,118,27437.367363,2283,962
3,Boston,2018-02-19 15:00:00,5 mins,True,True,1984,3584,355,44995.452993,4035,2021
4,Boston,2018-02-19 17:40:00,2 mins,False,False,1371,2580,181,27583.955295,2200,979


In [6]:
df_raw['total_ride'] = df_raw['trips_express_pool'] + df_raw['trips_pool']
df_raw['cost_per_ride'] = df_raw['total_driver_payout']/df_raw['total_ride']
df_raw['double_matches_rate'] = df_raw['total_double_matches']/df_raw['total_ride']
df_raw['match_rate'] = df_raw['total_matches']/df_raw['total_ride']

In [90]:
df_raw.head()

,city_id,period_start,wait_time,treat,commute,trips_pool,trips_express_pool,rider_cancellations,total_driver_payout,total_matches,total_double_matches,total_ride,cost_per_ride,double_matches_rate,match_rate
0,Boston,2018-02-19 07:00:00,2 mins,False,True,1415,3245,256,34458.411634,3372,1476,4660,7.394509,0.316738,0.723605
1,Boston,2018-02-19 09:40:00,5 mins,True,False,1461,2363,203,29764.349821,2288,1275,3824,7.783564,0.333421,0.598326
2,Boston,2018-02-19 12:20:00,2 mins,False,False,1362,2184,118,27437.367363,2283,962,3546,7.737554,0.271292,0.643824
3,Boston,2018-02-19 15:00:00,5 mins,True,True,1984,3584,355,44995.452993,4035,2021,5568,8.081080,0.362967,0.724677
4,Boston,2018-02-19 17:40:00,2 mins,False,False,1371,2580,181,27583.955295,2200,979,3951,6.981512,0.247785,0.556821


#### Question 1

What is the difference between ride cancellations and cost per trip of 2 min wait time against a 5 min wait time? Is the difference statistically significant at a 95% confidence level (5% significant level)? Write one-sided hypothesis tests and use parametric test statistics (you don’t need to validate the assumption of the parametric approach).

In [9]:
summary = df_raw.groupby(['wait_time'])['rider_cancellations','cost_per_ride'].agg('mean').T
summary['diff (2mins - 5mins)'] = summary['2 mins'] - summary['5 mins']
summary

<ipython-input-9-4a786d50c656>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  summary = df_raw.groupby(['wait_time'])['rider_cancellations','cost_per_ride'].agg('mean').T


wait_time,2 mins,5 mins,diff (2mins - 5mins)
rider_cancellations,165.349206,190.126984,-24.777778
cost_per_ride,7.364866,6.987458,0.377409


##### Ride Cancellation

$Hypothesis Test\\
 H_0: \mu_{ride\ cancellations\ in\ 2min} - \mu_{ride\ cancellations\ in\ 5min} \ge 0\\ 
 H_a: \mu_{ride\ cancellations\ in\ 2min} - \mu_{ride\ cancellations\ in\ 5min} < 0$


In [17]:
statistic,p_value =  ttest_ind(df_raw[df_raw['treat']==False]['rider_cancellations'],df_raw[df_raw['treat']==True]['rider_cancellations'], alternative='less')
print('Statistics = %.3f, P-value=%.3f' %(statistic,p_value))

Statistics = -2.764, P-value=0.003


- Finding: T-test: P-value=0.03 < $α$ = 0.05, we reject  the null hypothesis
- Conclusion: We are 95% confident to reject the null hypothesis that the differnece betweeen ride cancellations of 2 min wait time against a 5 min wait time is greater or equal to zero. Instead, rider cancellations of 2 min wait time are roughly 25 times less than that of 5 min wait time. 

##### Cost per Trip 

$Hypothesis Test\\
 H_0: \mu_{cost\ per\ trip\ in\ 2min} - \mu_{cost\ per\ trip\ in\ 5min} \le 0\\ 
 H_a: \mu_{cost\ per\ trip\ in\ 2min} - \mu_{cost\ per\ trip\ in\ 5min}> 0$


In [18]:
statistic,p_value =  ttest_ind(df_raw[df_raw['treat']==False]['cost_per_ride'],df_raw[df_raw['treat']==True]['cost_per_ride'], alternative='greater')
print('Statistics = %.3f, P-value=%.3f' %(statistic,p_value))

Statistics = 3.462, P-value=0.000


- Finding: T-test: P-value=0.000 < $α$ = 0.05, we reject  the null hypothesis
- Conclusion: We are 95% confident to reject the null hypothesis that the differnece betweeen ride cancellations of 2 min wait time against a 5 min wait time is less or equal to zero. Instead, cost per ride of 2 min wait time are roughly $0.38 more than that of 5 min wait time. 

#### Question 2

Does the median number of express trips,  ride cancellations, and cost per trip change for commuting versus non-commuting hours at a 95% level of confidence (5%  significant level)? Use a one-sided hypothesis test to confirm your findings (Use only non-parametric test statistics)

In [50]:
df_commute = df_raw[df_raw['commute']==True]
df_non_commute = df_raw[df_raw['commute']==False]

In [55]:
# summary of commute
summary = df_commute.groupby(['wait_time'])['rider_cancellations','cost_per_ride','trips_express_pool'].agg('median').T
summary['diff (2mins - 5mins)'] = summary['2 mins'] - summary['5 mins']
summary

<ipython-input-55-c1b9199226b8>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  summary = df_commute.groupby(['wait_time'])['rider_cancellations','cost_per_ride','trips_express_pool'].agg('median').T


wait_time,2 mins,5 mins,diff (2mins - 5mins)
rider_cancellations,240.500000,290.000000,-49.500000
cost_per_ride,7.987099,7.647872,0.339226
trips_express_pool,3434.500000,3280.500000,154.000000


In [56]:
# summary of non-commute
summary = df_non_commute.groupby(['wait_time'])['rider_cancellations','cost_per_ride','trips_express_pool'].agg('median').T
summary['diff (2mins - 5mins)'] = summary['2 mins'] - summary['5 mins']
summary

<ipython-input-56-b9fc4c55b17b>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  summary = df_non_commute.groupby(['wait_time'])['rider_cancellations','cost_per_ride','trips_express_pool'].agg('median').T


wait_time,2 mins,5 mins,diff (2mins - 5mins)
rider_cancellations,156.000000,170.000000,-14.000000
cost_per_ride,7.225132,6.881117,0.344015
trips_express_pool,2453.000000,2276.000000,177.000000


##### Number of Express Pool During Commute Period

$Hypothesis\ Test\\
 H_0: median_{num\ of\ express\ pool\ in\ 2\ mins} \le median_{num\ of\ express\ pool\ in\ 5\ mins}\\ 
 H_0: median_{num\ of\ express\ pool\ in\ 2\ mins} > median_{num\ of\ express\ pool\ in\ 5\ mins}\\ $


In [69]:
statistic,p_value =  stats.mannwhitneyu(df_commute[df_commute['treat']==False]['trips_express_pool'], df_commute[df_commute['treat']==True]['trips_express_pool'], alternative='greater')
print('Statistics = %.3f, P-value=%.3f' %(statistic,p_value)) 

Statistics = 68.000, P-value=0.093


- Finding: T-test: P-value=0.093 > $α$ = 0.05, we fail reject  the null hypothesis
- Conclusion: Using the 5% siginificane level, we fail to  reject the null hypothesis that the median number of express pool of 2 min wait time is less or equal to that of a 5 min wait time.

##### Number of Express Pool During Non-Commute Period

$Hypothesis\ Test\\
 H_0: median_{num\ of\ express\ pool\ in\ 2\ mins} \le median_{num\ of\ express\ pool\ in\ 5\ mins}\\ 
 H_0: median_{num\ of\ express\ pool\ in\ 2\ mins} > median_{num\ of\ express\ pool\ in\ 5\ mins}\\ $


In [70]:
statistic,p_value =  stats.mannwhitneyu(df_non_commute[df_non_commute['treat']==False]['trips_express_pool'], df_non_commute[df_non_commute['treat']==True]['trips_express_pool'], alternative='greater')
print('Statistics = %.3f, P-value=%.3f' %(statistic,p_value)) 

Statistics = 1838.500, P-value=0.003


- Finding: T-test: P-value=0.003 < $α$ = 0.05, we reject  the null hypothesis
- Conclusion: We are 95% confident to reject the null hypothesis that the median number of express pool of a 2 min wait time is less or equal to that of a 5 min wait time. Instead, the median number of epress pool of a 2 min wait time are roughly 177 times more than that of 5 min wait time. 

##### Number of Rider Cancellations During Commute Period

$Hypothesis\ Test\\
 H_0: median_{num\ of\ rider\ cancellations\ in\ 2\ mins} \ge median_{num\ of\ rider\ cancellations\ in\ 5\ mins}\\ 
 H_0: median_{num\ of\ rider\ cancellations\ in\ 2\ mins} < median_{num\ of\ rider\ cancellations\ in\ 5\ mins}\\ $


In [71]:
statistic,p_value =  stats.mannwhitneyu(df_commute[df_commute['treat']==False]['rider_cancellations'], df_commute[df_commute['treat']==True]['rider_cancellations'], alternative='less')
print('Statistics = %.3f, P-value=%.3f' %(statistic,p_value)) 

Statistics = 13.000, P-value=0.003


- Finding: T-test: P-value=0.003 < $α$ = 0.05, we reject  the null hypothesis
- Conclusion: We are 95% confident to reject the null hypothesis that the median number of rider cancellations of a 2 min wait time is greater or equal to that of a 5 min wait time. Instead, the median number of rider cancellations of a 2 min wait time are roughly 50 times less than that of 5 min wait time. 

##### Number of Rider Cancellations During Non-Commute Period

$Hypothesis\ Test\\
 H_0: median_{num\ of\ rider\ cancellations\ in\ 2\ mins} \ge median_{num\ of\ rider\ cancellations\ in\ 5\ mins}\\ 
 H_0: median_{num\ of\ rider\ cancellations\ in\ 2\ mins} < median_{num\ of\ rider\ cancellations\ in\ 5\ mins}\\ $


In [73]:
statistic,p_value =  stats.mannwhitneyu(df_non_commute[df_non_commute['treat']==False]['rider_cancellations'], df_non_commute[df_non_commute['treat']==True]['rider_cancellations'], alternative='less')
print('Statistics = %.3f, P-value=%.3f' %(statistic,p_value)) 

Statistics = 841.500, P-value=0.000


- Finding: T-test: P-value=0.000 < $α$ = 0.05, we reject  the null hypothesis
- Conclusion: We are 95% confident to reject the null hypothesis that the median number of rider cancellations of a 2 min wait time is greater or equal to that of a 5 min wait time. Instead, the median number of rider cancellations of a 2 min wait time are roughly 14 times less than that of 5 min wait time. 

##### Cost per Trip During Commute Period

$Hypothesis\ Test\\
 H_0: median_{cost\ per\ trip\ in\ 2\ mins} \le median_{cost\ per\ trip\ in\ 5\ mins}\\ 
 H_0: median_{cost\ per\ trip\ in\ 2\ mins} > median_{cost\ per\ trip\ in\ 5\ mins}\\ $


In [84]:
statistic,p_value =  stats.mannwhitneyu(df_commute[df_commute['treat']==False]['cost_per_ride'], df_commute[df_commute['treat']==True]['cost_per_ride'], alternative='greater')
print('Statistics = %.3f, P-value=%.3f' %(statistic,p_value)) 

Statistics = 65.000, P-value=0.137


- Finding: T-test: P-value=0.003 < $α$ = 0.05, we reject  the null hypothesis
- Conclusion: at the 5% significance level, we fail to reject the null hypothesis that the median cost per trip in a 2 min wait time is less than or equal to that of a 5 min wait time.

##### Cost per Trip During Non-Commute Period

$Hypothesis\ Test\\
 H_0: median_{cost\ per\ trip\ in\ 2\ mins} \le median_{cost\ per\ trip\ in\ 5\ mins}\\ 
 H_0: median_{cost\ per\ trip\ in\ 2\ mins} > median_{cost\ per\ trip\ in\ 5\ mins}\\ $


In [85]:
statistic,p_value =  stats.mannwhitneyu(df_non_commute[df_non_commute['treat']==False]['cost_per_ride'], df_non_commute[df_non_commute['treat']==True]['cost_per_ride'], alternative='greater')
print('Statistics = %.3f, P-value=%.3f' %(statistic,p_value)) 

Statistics = 1935.000, P-value=0.000


- Finding: T-test: P-value=0.000 < $α$ = 0.05, we reject  the null hypothesis
- Conclusion: We are 95% confident to reject the null hypothesis that the median number of cost per trip of a 2 min wait time is less than or equal to that of a 5 min wait time. Instead, the median number of cost per trip of a 2 min wait time are roughly $0.344 more than that of 5 min wait time. 

#### Question 3

Assumptions
1. Total ride in a 5 min wait time would be as many as that in a 2 min wait time, if the 2 min wait time is replaced by the 5 min wait time.
2. The average number of passengers who take Uber in the coming three weeks will be roughly the same as that in the past.
3. The average difference of payout to driver in a 2 min wait time and in a 5 min wait time in the coming three weeks will be roughly the same as that in the past.
4. The average number of passeners, in the rest of the cities, who take Uber will be roughly the same as that in Boston.

In [103]:
summary = df_raw.groupby(['wait_time'])['total_ride','cost_per_ride'].agg('mean').T
summary

<ipython-input-103-a3f0069257a6>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  summary = df_raw.groupby(['wait_time'])['total_ride','cost_per_ride'].agg('mean').T


wait_time,2 mins,5 mins
total_ride,3966.984127,3880.079365
cost_per_ride,7.364866,6.987458


In [104]:
cost_lost_per_day_per_city = round((summary['2 mins']['total_ride'])*(summary['2 mins']['cost_per_ride']-summary['5 mins']['cost_per_ride'])*9)
cost_lost_per_day_per_city

13475

In [105]:
total_cost_in_dollar = round(cost_lost_per_day_per_city * 3 * 7 * 6)
total_cost_in_dollar

1697850

- Conclusion: Under the above assumptions, Uber would lose roughly $1.7 million in the next three weeks at the 6 cities, if wait time were kept at 2 minutes to 5 minutes.

#### Question 4

Recommendation:

Based on the available data, I would recommend Uber continue with the 2 minute wait time option and not increase the wait time to 5 minute in the six treatment citiies. 
1. The average number of cancellations of a 2 min wait time, during a 160-min interval, is 25 times lower than that of a 5 min wait time. The average number of cancellations is even lower during commute period. This could increase both customer and driver satisfcation, while lower the matching cost of Uber. lower cancellation can also prevent potential churning of customers. 
2. The average number of express pool of a 2 min wait can is greater than that of a 5 min wait time, especially during commute period. This could bring more revenue to Uber.
3. Though the cost of the 2 min wait is higher than that of the 5 min wait time, it can bring more passengers and lower cancellation rate, which mitigates the cost.

In the future: 
1. If the price per trip could be provided, profit could be calculated. Decisions made on profit would be accurate.
2. If customer and driver's rate could be provided, long-term benefit of lauching the Express Pool in different wait time could be quantified.